# Trump Twitter Analysis



In [1]:
import os
import numpy as np
import tensorflow as tf
import pandas as pd

In [2]:
# configs
DATAFILE = 'trump_tweets_11_17.json'

In [3]:
# read in json file as pd dataframe
trump_tweets = pd.read_json(DATAFILE)
print('number of rows: %d' % trump_tweets.shape[0])

number of rows: 20293


In [4]:
# id_str column is unique
print(trump_tweets['id_str'].is_unique)

# set id_str as unique identifier
trump_tweets = trump_tweets.set_index('id_str')

True


In [5]:
trump_tweets.head()

,source,text,created_at,retweet_count,favorite_count,is_retweet
id_str,,,,,,
1196148537525977088,Twitter for iPhone,.@SteveScalise blew the nasty &amp; obnoxious ...,2019-11-17 19:30:09+00:00,2663,8670,0.0
1196147508445163520,Twitter for iPhone,.@SteveScalese blew the nasty &amp; obnoxious ...,2019-11-17 19:26:04+00:00,789,2384,0.0
1196134600227078144,Twitter for iPhone,Thanks Eric! https://t.co/6Ai7bqto3P,2019-11-17 18:34:46+00:00,4349,14897,0.0
1196134448183566336,Twitter for iPhone,RT @EricTrump: I hope no one who serves our co...,2019-11-17 18:34:10+00:00,16239,0,1.0
1196134298325266432,Twitter for iPhone,RT @EricTrump: Trump Washington D.C! @TrumpDC ...,2019-11-17 18:33:34+00:00,2505,0,1.0


In [6]:
# remove rows that aren't iPhone/Android
trump_tweets = trump_tweets[trump_tweets.source.isin(['Twitter for Android', 'Twitter for iPhone'])]
print('number of tweets from iPhone + Android:\t%d' % trump_tweets.shape[0])

number of tweets from iPhone + Android:	18599


In [7]:
trump_tweets.head()

,source,text,created_at,retweet_count,favorite_count,is_retweet
id_str,,,,,,
1196148537525977088,Twitter for iPhone,.@SteveScalise blew the nasty &amp; obnoxious ...,2019-11-17 19:30:09+00:00,2663,8670,0.0
1196147508445163520,Twitter for iPhone,.@SteveScalese blew the nasty &amp; obnoxious ...,2019-11-17 19:26:04+00:00,789,2384,0.0
1196134600227078144,Twitter for iPhone,Thanks Eric! https://t.co/6Ai7bqto3P,2019-11-17 18:34:46+00:00,4349,14897,0.0
1196134448183566336,Twitter for iPhone,RT @EricTrump: I hope no one who serves our co...,2019-11-17 18:34:10+00:00,16239,0,1.0
1196134298325266432,Twitter for iPhone,RT @EricTrump: Trump Washington D.C! @TrumpDC ...,2019-11-17 18:33:34+00:00,2505,0,1.0


In [8]:
# extract target array
target_array = trump_tweets.pop('source')

In [9]:
del trump_tweets['created_at']

In [10]:
trump_tweets.text = trump_tweets.text.astype(str)

In [11]:
trump_tweets.head()

,text,retweet_count,favorite_count,is_retweet
id_str,,,,
1196148537525977088,.@SteveScalise blew the nasty &amp; obnoxious ...,2663,8670,0.0
1196147508445163520,.@SteveScalese blew the nasty &amp; obnoxious ...,789,2384,0.0
1196134600227078144,Thanks Eric! https://t.co/6Ai7bqto3P,4349,14897,0.0
1196134448183566336,RT @EricTrump: I hope no one who serves our co...,16239,0,1.0
1196134298325266432,RT @EricTrump: Trump Washington D.C! @TrumpDC ...,2505,0,1.0


In [12]:
trump_tweets = trump_tweets.text

In [13]:
trump_tweets.head()

id_str
1196148537525977088    .@SteveScalise blew the nasty &amp; obnoxious ...
1196147508445163520    .@SteveScalese blew the nasty &amp; obnoxious ...
1196134600227078144                 Thanks Eric! https://t.co/6Ai7bqto3P
1196134448183566336    RT @EricTrump: I hope no one who serves our co...
1196134298325266432    RT @EricTrump: Trump Washington D.C! @TrumpDC ...
Name: text, dtype: object

In [14]:
# import data
dataset = tf.data.Dataset.from_tensor_slices((trump_tweets.values, target_array.values))

In [15]:
# shuffle
train_dataset = dataset.shuffle(len(trump_tweets)).batch(1)

In [16]:
train_dataset

<BatchDataset shapes: ((None,), (None,)), types: (tf.string, tf.string)>